In [1]:
! pip install openvino==2021.4.2 openvino-dev==2021.4.2

  Using cached numpy-1.19.5-cp39-cp39-macosx_10_9_x86_64.whl (15.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.2


    Uninstalling numpy-1.22.2:
      Successfully uninstalled numpy-1.22.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
You should consider upgrading via the '/Users/AlexHome/vms/effdl-inference/venv/bin/python -m pip install --upgrade pip' command.


In [24]:
import torchvision.models as models
import torch

vgg16 = models.vgg16(pretrained=True)

example = torch.rand(1, 3, 224, 224)
torch.onnx.export(vgg16, example, "vgg16.onnx", verbose=False)

In [25]:
! mo --input_model vgg16.onnx --output_dir vgg16.vino

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/Users/AlexHome/vms/effdl-inference/07-software-inference/02-openvino/vgg16.onnx
	- Path for generated IR: 	/Users/AlexHome/vms/effdl-inference/07-software-inference/02-openvino/vgg16.vino
	- IR output name: 	vgg16
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	False
ONNX specific parameters:
	- Inference Engine found in: 	/Users/AlexHome/vms/effdl-inference/venv/lib/python3.9/site-packages/openvino
Inference Engine version: 	2021.4.2-3976-0943ed67223-re

In [26]:
! ls vgg16.vino/

vgg16.bin     vgg16.mapping vgg16.xml


In [5]:
! head -n 100 vgg16.vino/vgg16.xml

<?xml version="1.0" ?>
<net name="vgg16" version="10">
	<layers>
		<layer id="0" name="input.1" type="Parameter" version="opset1">
			<data shape="1, 3, 224, 224" element_type="f32"/>
			<output>
				<port id="0" precision="FP32" names="input.1">
					<dim>1</dim>
					<dim>3</dim>
					<dim>224</dim>
					<dim>224</dim>
				</port>
			</output>
		</layer>
		<layer id="1" name="features.0.weight" type="Const" version="opset1">
			<data element_type="f32" shape="64, 3, 3, 3" offset="0" size="6912"/>
			<output>
				<port id="0" precision="FP32" names="features.0.weight">
					<dim>64</dim>
					<dim>3</dim>
					<dim>3</dim>
					<dim>3</dim>
				</port>
			</output>
		</layer>
		<layer id="2" name="Conv_0/WithoutBiases" type="Convolution" version="opset1">
			<data strides="1, 1" dilations="1, 1" pads_begin="1, 1" pads_end="1, 1" auto_pad="explicit"/>
			<input>
				<port id="0" precision="FP32">
					<dim>1</dim>
					<dim>3</dim>
					<dim>224</dim>
					<dim>224</dim>
				</port>
		

In [6]:
from openvino.inference_engine import IECore

ie = IECore()

In [7]:
devices = ie.available_devices
for device in devices:
    device_name = ie.get_metric(device_name=device, metric_name="FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")

CPU: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz


In [8]:
devices

['CPU']

In [9]:
net = ie.read_network(model="vgg16.vino/vgg16.xml")
exec_net = ie.load_network(network=net, device_name="CPU")

In [10]:
net.input_info

{'input.1': <openvino.inference_engine.ie_api.InputInfoPtr at 0x110a9b5d0>}

In [11]:
input_layer = next(iter(net.input_info))
input_layer

'input.1'

In [12]:
print(f"input layout: {net.input_info[input_layer].layout}")
print(f"input precision: {net.input_info[input_layer].precision}")
print(f"input shape: {net.input_info[input_layer].tensor_desc.dims}")

input layout: NCHW
input precision: FP32
input shape: [1, 3, 224, 224]


In [13]:
output_layer = next(iter(net.outputs))
print(f"output layout: {net.outputs[output_layer].layout}")
print(f"output precision: {net.outputs[output_layer].precision}")
print(f"output shape: {net.outputs[output_layer].shape}")

output layout: NC
output precision: FP32
output shape: [1, 1000]


In [14]:
import torchvision.transforms as transforms

transform_pipeline = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])


def prepare(img_path):
    image = Image.open(img_path)
    img_data = transform_pipeline(image).unsqueeze(0)
    data = np.array(img_data)
    return data

In [15]:
input_layer

'input.1'

In [16]:
import numpy as np
from IPython.display import display
from PIL import Image

img = prepare('1200px-Cat03.jpg')

In [18]:
input_layer

'input.1'

In [19]:
result = exec_net.infer({input_layer: img})
result

{'70': array([[-5.11787319e+00, -9.00457442e-01, -1.33539951e+00,
         -3.37547809e-02, -1.62641668e+00,  1.24357975e+00,
         -1.14814985e+00,  1.55837834e-02,  2.63606358e+00,
         -3.83762908e+00, -1.54843235e+00, -2.95213914e+00,
          1.23569787e+00, -7.08230615e-01, -3.13737106e+00,
         -7.40746796e-01, -4.38675374e-01,  1.04632042e-01,
         -2.55209517e+00, -3.68814874e+00, -2.86074638e+00,
         -3.22845531e+00, -4.80303001e+00, -3.59836626e+00,
         -2.71968842e+00, -4.30782700e+00, -4.68665457e+00,
         -2.75465584e+00, -4.50009346e+00, -2.30202293e+00,
         -1.76955783e+00, -2.61921239e+00, -2.64150214e+00,
         -3.41213751e+00, -3.77243614e+00, -4.06072760e+00,
         -1.76341093e+00, -3.37075043e+00, -7.38503337e-01,
         -2.89529181e+00, -6.44332588e-01, -2.17674804e+00,
         -2.18159175e+00, -2.43516064e+00, -1.87657809e+00,
         -4.09389162e+00, -1.22118831e+00, -2.08098173e+00,
         -4.20933580e+00, -5.28374

In [20]:
values = next(iter(result.values()))

In [21]:
np.argmax(values)

285

In [23]:
! mkdir -p inference-server/models_repo/vgg16/1
! cp vgg16.vino/vgg16.bin inference-server/models_repo/vgg16/1/ir_model.bin
